# Objectifs :
- Extraire des infos intéressantes type :
    - prix min, moyen et max
    - durée min, max, moyenne par trajet
- Différence de prix moyen et durée selon le train, le bus et le covoit selon la distance du trajet
    
    Par exemple (0-200km, 201-800km, 800-2000km, 2000+km)
    
- Le plus d’infos bonus !
    
    Comme par exemple :
    
    *Graphes, prédictions de prix, rapport des soucis relevés dans les données, visualisation interactive, sourcing & utilisation de données externes pertinentes, utilisation d’API externes*

# Info utiles sur les fichiers CSV :
- ***ticket_data.csv*** : Contenant un historique de ticket (une ligne => une proposition de ticket sur tictactrip)
- ***cities.csv*** les villes desservies par tictactrip (lien grâce aux colonnes o_city (origin_city), d_city (destination_city) de ticket_data)
- ***stations.csv*** les stations desservies par tictactrip (lien via o_station, d_station de ticket_data)
- ***providers.csv*** infos sur les différents providers (lien via company de ticket_data)Un provider est une "sous-compagnie". Par exemple TGV et TER sont deux providers de VSC (voyages-sncf).

### 1- Import

In [ ]:
%load_ext autoreload
%autoreload 2
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn import metrics
from sklearn.model_selection import cross_validate, GridSearchCV, KFold

from process import *

### 2- Fouille de données

In [ ]:
TICKET_FILE = '../data/raw/ticket_data.csv'
PROVIDERS_FILE = '../data/raw/providers.csv'
STATIONS_FILE = '../data/raw/stations.csv'
CITIES_FILE = '../data/raw/cities.csv'
df = pd.read_csv(TICKET_FILE)
df_providers = pd.read_csv(PROVIDERS_FILE)
df_stations = pd.read_csv(STATIONS_FILE)
df_cities = pd.read_csv(CITIES_FILE)

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
len(df[df['arrival_ts'] < df['departure_ts']])

In [ ]:
df[df['departure_ts'] < df['search_ts']][['id', 'search_ts', 'departure_ts']]


Le service propose des tickets dont la date de départ est antérieur à celle de la date de la recherche.

In [ ]:
df.dtypes

In [ ]:
df_middle_stations = pd.DataFrame()
df_middle_stations['middle_stations'] = df['middle_stations'].copy()

df_middle_stations['middle_stations'] = df_middle_stations['middle_stations'].where(df_middle_stations['middle_stations'].isna(), "arrêts intermédiaires")
df_middle_stations['middle_stations'] = df_middle_stations['middle_stations'].where(df_middle_stations['middle_stations'].notna(), "ligne directe")
df_middle_stations["middle_stations"].value_counts().plot(kind='pie'
                                        , title="Proportion de trajets avec des arrêts intermédiaires ou non"
                                        , ylabel=''
                                        , xlabel=''
                                        , autopct='%.2f')
plt.show()


In [ ]:
df_copy = df[df['departure_ts'] > df['search_ts']].copy()
df_copy['search_ts'] = pd.to_datetime (df_copy['search_ts'])
df_copy['departure_ts'] = pd.to_datetime (df_copy['departure_ts'])
df_copy['arrival_ts'] = pd.to_datetime (df_copy['arrival_ts'])

df_copy = df_copy.sort_values(by='search_ts')

df_copy[['id', 'search_ts', 'departure_ts', 'arrival_ts']].head(5)


In [ ]:
df_copy[['id', 'search_ts', 'departure_ts', 'arrival_ts']].tail(5)

Les recherches ont été faite entre le 10/01/2017 et le 19/10/2017.

In [ ]:
df_copy['time_difference'] = df_copy['search_ts'].diff().dt.total_seconds()
print(df_copy['time_difference'].describe())

Il y a en moyenne une recherche toute les 21 secondes. <br/>
le plus grand intervalle de temps entre 2 recherches est de 27078.821 secondes soit environ 7h50min.

In [ ]:
df_plot = df_copy.set_index(df_copy['search_ts'].map(lambda s: s.strftime('%Y-%m-%d')))
fig, ax=plt.subplots(figsize=(10,10))
df_plot['time_difference'].plot(marker="o"
                                , linewidth=0
                                , x='search_ts'
                                , ylim=[0, df_copy['time_difference'].max()]
                                ,rot=30)

ax.set_ylabel("Différence de temps entre 2 recherches en secondes")
ax.set_xlabel("date de la recherche")
ax.set_title("Répartition des recherches en fonction du temps")
plt.show()

In [ ]:
df_copy['travel_time'] = (df_copy['arrival_ts'] - df_copy['departure_ts']).dt.total_seconds() / 3600
print(df_copy['travel_time'].describe())

In [ ]:
out = pd.cut(df_copy['travel_time']
            , bins=[0, 5, 10, 24, 1000]
            , include_lowest=True
            , labels=['moins de 5h', 'entre 5 et 10h', 'entre 10h et 1j', 'plus de 1j'])
out_norm = out.value_counts(sort=False, normalize=True) * 100

ax = out_norm.plot(kind='bar'
                , rot=0
                , figsize=(10,10))
ax.set_ylabel("nombres de voyages en pourcentage")
ax.set_xlabel("durée du trajet")
ax.set_title("Durée d'un voyage")
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
df_copy['price_in_cents'].describe()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax = df['price_in_cents'].plot(kind='hist'
                            , color='orange'
                            , alpha=0.5)
df['price_in_cents'].plot(kind='kde'
                            , title='Distribution du prix des tickets'
                            , ax=ax
                            , secondary_y=True
                            , color='blue'
                            , xlim=[df['price_in_cents'].min(), df['price_in_cents'].max()])
ax.set_xlabel("prix en centimes")

plt.show()

In [ ]:
companies = df_providers.query('provider_id.isnull()')[['company_id', 'fullname']]
df_companies = df_copy.apply(lambda x :get_company_name (x, df_providers, companies), axis=1)

In [ ]:
df_group_count_companies = df_companies.groupby(['company', 'id'], as_index=False).count()

In [ ]:
df_group_count_companies.company.value_counts()

Blablacar est la compagnie qui propose le plus de tickets.

In [ ]:
ax = df_group_count_companies.company.value_counts().plot(kind='bar'
                                            , title='Nombres de tickets en fonction de la compagnie'
                                            , figsize=(10,10))
ax.set_xlabel('Compagnie')
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.boxplot(x=df_companies['company']
                , y=df_companies['price_in_cents']
                , palette='hls')

ax.set_xticklabels(ax.get_xticklabels()
                , rotation=30)

plt.title('Prix du ticket en fonction de la compagnie')
plt.xlabel('Compagnie')
plt.ylabel('Prix')
plt.show()

Blablacar et Distribusion proposent les trajets les moins chers. <br/>
Voyages SNCF propose les prix les plus chers.

In [ ]:
df_copy = df_copy.apply(count_number_middle_stations, axis=1)

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.boxplot(x=df_copy['number_middle_stations']
                , y=df_companies['price_in_cents']
                , palette='hls')

plt.title("Prix du ticket en fonction du nombre d'arrêts")
plt.xlabel("Nombre d'arrêts")
plt.ylabel('Prix')
plt.show()

Les trajets avec 4 arrêts sont les plus chers.<br/>
Les trajets sans arrêts sont les moins chers.

In [ ]:
df_copy['days_left'] = (df_copy['departure_ts'] - df_copy['search_ts']).astype('timedelta64[D]')
print(df_copy['days_left'].describe())

In [ ]:
plt.figure(figsize=(20,8))
sns.lineplot(data=df_copy,x='days_left',y='price_in_cents',color='blue')
plt.title('Prix du ticket en fonction du nombre de jours avant le départ')
plt.xlabel('Nombre de jours avant le départ')
plt.ylabel('Prix')
plt.show()

In [ ]:
bins_distance = [0, 200, 800, 2000, 10000]

In [ ]:
df_copy = df_copy.apply(lambda x: get_time_of_day(x, 'departure_ts'), axis=1)
df_copy = df_copy.apply(lambda x: get_time_of_day(x, 'arrival_ts'), axis=1)

In [ ]:
df_copy = df_copy.apply(lambda x: get_distance_between_cities(x, df_cities), axis=1)

In [ ]:
df_copy = df_copy.apply(lambda x: get_info_providers(x, df_providers), axis=1)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
sns.boxplot(x='departure_ts_time_of_day',y='price_in_cents',data=df_copy)
plt.title("Prix en fonction de l'heure de départ")
plt.xlabel('Départ - Moment de la journée')
plt.ylabel('Prix')

plt.subplot(1,2,2)
sns.boxplot(x='arrival_ts_time_of_day',y='price_in_cents',data=df_copy)
plt.title("Prix en fonction de l'heure d'arrivée")
plt.xlabel('Arrivée - Moment de la journée')
plt.ylabel('')
plt.show()

Le prix du trajet est plus cher quand on souhaite partir la nuit. <br/>
Le prix du trajet est le moins cher quand on souhaite partir l'après-midi ou le soir. <br/>
Le prix du trajet à tendance à être plus élevé quand on souhaite arriver le matin. <br/> 
Le prix du trajet à tendance à être légèrement moins cher quand on souhaite arriver le soir. <br/>

In [ ]:
df_transport_mean_price = df_copy.groupby (by=['transport_type', pd.cut(df_copy['distance'], bins_distance, labels=['0-200km', '200-800km', '800-2000km', '2000km+'])])['price_in_cents'].mean()
df_transport_mean_price.unstack()

In [ ]:
df_transport_mean_travel_time = df_copy.groupby (by=['transport_type', pd.cut(df_copy['distance'], bins_distance, labels=['0-200km', '200-800km', '800-2000km', '2000km+'])])['travel_time'].mean()
df_transport_mean_travel_time.unstack()

In [ ]:
df_copy['has_wifi'] = df_copy['has_wifi'].replace ({True:1, False:0})
df_copy['has_plug'] = df_copy['has_plug'].replace ({True:1, False:0})
df_copy['has_adjustable_seats'] = df_copy['has_adjustable_seats'].replace ({True:1, False:0})
df_copy['has_bicycle'] = df_copy['has_bicycle'].replace ({True:1, False:0})

In [ ]:
df_dummies_transport = pd.get_dummies(df_copy['transport_type'])
df_corr = pd.concat([df_copy, df_dummies_transport], axis=1)
del df_corr['transport_type']

In [ ]:
df_corr.corr()

Nous allons conserver les colonnes qui possèdent une corrélation supérieure à 0.15 (en valeur absolue).
Les colonnes conservées sont donc :
- travel_time
- distance
- has_adjustable_seats
- has_bicycle
- carpooling
- train
- number_middle_stations

In [ ]:
df_to_plot = df_corr[['travel_time', 'distance', 'has_adjustable_seats', 'has_bicycle', 'carpooling', 'train', 'number_middle_stations', 'price_in_cents']]
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.heatmap(df_to_plot.corr()
                , vmin=-1, vmax=1, center=0
                , cmap=sns.diverging_palette(20, 220, n=200)
                , square=True
                , annot=True
)
ax.set_xticklabels(ax.get_xticklabels()
                , rotation=30
                , horizontalalignment='right')
ax.set_title ("matrice de corrélations");

La matrice des corrélations affiche des valeurs (corrélations) comprise entre -1 et 1. Lorsque la valeur est proche de +-1, les 2 variables sont dites fortement corrélées. Si la valeur est proche de 0 alors les 2 variables ne sont pas corrélées. <br/>
Pour la ligne du prix on peut lire par exemple :
- Lorsque la distance du trajet augmente alors le prix du trajet a tendance à monter.
- Lorsqu'il y a un rangement pour vélo alors le prix du trajet a tendance à monter.
- Lorsque le type de trajet est "carpooling" alors le prix du trajet a tendance à diminuer. <br/>


### Analyses
- id : numéro du ticket
- company : société qui propose le service
- o_station : station d'origine desservie
- d_station : station de destination desservie
- departure_ts : date de départ
- arrival_ts : date d'arrivée
- price_in_cents : prix en centimes
- search_ts : date de la recherche (date à laquelle le trajet a été proposé sur l'application)
- middle_stations : stations intermédiaires
- other_companies : autres sociétés
- o_city : ville de départ
- d_city : ville d'arrivée

o_city et d_city correspondent respectivement au point de départ et au point d'arrivée d'un trajet (de A -> B). On peut retrouver des infos sur ces points dans le fichier cities. <br/>
o_station et d_stations correspondent aux stations desservies. On peut retrouver sdes infos sur ces stations dans le fichier stations.<br/>
On remarque des valeurs NaN sur 4 colonnes : o_station, d_station, middle_stations, other_companies.<br/>
Ces 4 colonnes sont corrélées à la société de transport. En effet, on remarque que lorsque la société propose des voitures en tant que moyen de transport (ligne 0), les 4 colonnes semblent avoir la valeur NaN.<br/>
De même, lorsque la société propose un train en tant que moyen de transport (ligne 2), les 4 colonnes semblent avoir des valeurs.<br/>
Cette observation parait logique puisque un voyage en train a pour habitude d'avoir des arrêts (sauf train direct) alors que faire du covoiturage limite les chances d'avoir des arrêts intermédiaires.

### Data Preparation

In [ ]:
df_regression = df_corr[['travel_time', 'distance', 'has_adjustable_seats', 'has_bicycle', 'carpooling', 'train', 'price_in_cents']]
df_regression = df_regression.fillna(0)

In [ ]:
df_regression[['travel_time', 'distance']].plot(subplots=True, layout=(1,2), figsize=(10,10))
plt.show()

In [ ]:
df_regression

In [ ]:
X_train, X_test, Y_train, Y_test = create_train_test_set(df_regression)

### Training

In [ ]:
NUMBER_FOLDS = 10
SEED = 42
kfolds = KFold(n_splits=NUMBER_FOLDS, random_state=SEED, shuffle=True)

models= {'LinearRegression' : LinearRegression()
        , 'DecisionTreeRegressor' : DecisionTreeRegressor()
        , 'RandomForestRegressor' : RandomForestRegressor()
        , 'Ridge' : Ridge()
        , 'Lasso' : Lasso()
}
scores = {}
scoring = ['neg_root_mean_squared_error', 'neg_mean_squared_error', 'neg_mean_absolute_error']
for key in models.keys():
        score = cross_validate(models[key], X_train, Y_train, cv=kfolds, scoring=scoring, return_estimator=True)
        scores[key] = score

In [ ]:
RMSE_values = {}
for key in scores.keys():
    print (key)
    print('Validation Mean Squared Error:', round(scores[key]['test_neg_mean_squared_error'].mean(), 2)) 
    print("Validation RMSE: ",round(scores[key]['test_neg_root_mean_squared_error'].mean(),2))
    print("Validation MAE: ",round(scores[key]['test_neg_mean_absolute_error'].mean(),2))
    print(' ')
    RMSE_values[key] = round(scores[key]['test_neg_root_mean_squared_error'].mean(),2)

best_model = min(RMSE_values, key=lambda x:abs(0 - RMSE_values[x]))
print (f"Meilleur modèle en comparant le RMSE : {best_model}")


In [ ]:
parameters_LR = {}

parameters_DTR = {"max_depth" : [1, 3, 5]
                , "min_samples_leaf":[1, 2, 3]
                , "max_leaf_nodes":[None, 10, 20, 30]}

parameters_RFR = {"max_depth" : [1, 3, 5]
                , "min_samples_split" : [2, 4, 6]
                , "min_samples_leaf":[1, 2, 3]}

parameters_Ridge = {'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}

parameters_Lasso = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03]}

parameters = {'LinearRegression' : parameters_LR
        , 'DecisionTreeRegressor' : parameters_DTR
        , 'RandomForestRegressor' : parameters_RFR
        , 'Ridge' : parameters_Ridge
        , 'Lasso' : parameters_Lasso
}

tuning_model = GridSearchCV(models[best_model]
                        , param_grid=parameters[best_model]
                        , scoring=scoring
                        , cv=kfolds
                        , refit='neg_root_mean_squared_error')
tuning_model.fit(X_train, Y_train)
print (tuning_model.best_params_)

### Evaluation

In [ ]:
if best_model == 'LinearRegression':
    tuned_model = LinearRegression(**tuning_model.best_params_)
    model = LinearRegression()
elif best_model == 'DecisionTreeRegressor':
    tuned_model = DecisionTreeRegressor(**tuning_model.best_params_)
    model = DecisionTreeRegressor()
elif best_model == 'RandomForestRegressor':
    tuned_model = RandomForestRegressor(**tuning_model.best_params_)
    model = RandomForestRegressor()
elif best_model == 'Ridge':
    tuned_model = Ridge(**tuning_model.best_params_)
    model = Ridge()
elif best_model == 'Lasso':
    tuned_model = Lasso(**tuning_model.best_params_)
    model = Lasso()

tuned_model.fit(X_train, Y_train)
Y_train_pred = tuned_model.predict(X_train)
Y_test_pred = tuned_model.predict(X_test)

model.fit(X_train, Y_train)
Y_train_pred_2 = model.predict(X_train)
Y_test_pred_2 = model.predict(X_test)

In [ ]:
print ('Score AVEC hyperparametres: ')
print('Training Mean Squared Error:', round(metrics.mean_squared_error(Y_train,Y_train_pred), 2)) 
print("Training RMSE: ", round(np.sqrt(metrics.mean_squared_error(Y_train,Y_train_pred)),2))
print("Training MAE: " ,round(metrics.mean_absolute_error(Y_train,Y_train_pred),2))

print(' ')

print('Test Mean Squared Error:', round(metrics.mean_squared_error(Y_test,Y_test_pred), 2)) 
print("Test RMSE: ", round(np.sqrt(metrics.mean_squared_error(Y_test,Y_test_pred)),2))
print("Test MAE: ", round(metrics.mean_absolute_error(Y_test,Y_test_pred),2))

print(' ')

print ('Score SANS hyperparametres: ')
print('Training Mean Squared Error:', round(metrics.mean_squared_error(Y_train,Y_train_pred_2), 2)) 
print("Training RMSE: ", round(np.sqrt(metrics.mean_squared_error(Y_train,Y_train_pred_2)),2))
print("Training MAE: " ,round(metrics.mean_absolute_error(Y_train,Y_train_pred_2),2))

print(' ')

print('Test Mean Squared Error:', round(metrics.mean_squared_error(Y_test,Y_test_pred_2), 2)) 
print("Test RMSE: ", round(np.sqrt(metrics.mean_squared_error(Y_test,Y_test_pred_2)),2))
print("Test MAE: ", round(metrics.mean_absolute_error(Y_test,Y_test_pred_2),2))

Ces 3 métriques représentent l'erreur moyenne entre la valeur originale et la valeur prédite. Une erreur de 0 signifie que le modèle est parfait. <br/>
Pour sélectionner le modèle, on cherche à minimiser ces métriques. <br/>
Le modèle sans l'hyperparamétrage est meilleur.

In [ ]:
df_pred = pd.DataFrame(Y_test.values, columns=['Verite'], index=Y_test.index)
df_pred['Prediction'] = Y_test_pred_2
df_pred = df_pred.reset_index()
df_pred

In [ ]:
df_pred[['Verite', 'Prediction']].plot(figsize=(15,10)
                                        , title='Comparaison des valeurs prédites avec les vraies valeurs'
                                        , xlabel='index de la valeur prédite'
                                        , ylabel='prix')
plt.show()

In [ ]:
df_copy.to_csv('../data/cleaned/ticket_data_cleaned.csv')